## Process Description

![Dietician](Assets/Simpy_clinic.png)

Simulate a process of dietary consultation. People visit the clinic, register at the front desk and then consult the dietician.

1. **Entity**: People arriving for dietary consultation
2. **Generator**: Generate arrivals for dietary consultation 
3. **Inter-Arrrival Time**: An exponential distribution sampled for patient arrivals  
4. **Activity**: Registration followed by consultation with dietician
5. **Activity Time**: Exponential distributions
6. **Resources**: Receptionist, dietician
7. **Queues**: People waiting for registration after arrivel and for consultation after registration
8. **Sink**: Exit after consultation

Based on: https://youtu.be/jXDjrWKcu6w

### 1. Import packages

Aside from **simpy**, we need **random** to sample from random distributions.

In [44]:
import simpy
import random

### 2. Generate Arrivals

Define the generator for arrivals. This puts patients in the queue for consultation.

**Args**:
- env (Simpy environment) required
- mean_IAT (int) mean Inter-Arrival Time for generating arrivals
- meanCT2register (int) mean Cycle Time for registration
- meanCT2consult (int) mean Cycle Time for consultation
- mean_CT (int) mean consultation time
- receptionist (Simpy resource) resource
- dietician (Simpy resource) resource

**Notes**:
- Has a forever-while loop in which it:
	- Creates a patient ID and sends the patient onward on their journey, invoking the function that generates consultations with the patient's ID.
	- Has one ```yield``` statement to timeout for sampled Inter-Arrival Time, after which it increments patient ID. The ```yield``` with timeout will await the action (i.e. completion of timer), freezing until then and next resuming where it left off.

In [45]:
def generate_arrivals(env, mean_IAT, meanCT2register, meanCT2consult, receptionist, dietician):
    # Create patient zero
    patient_ID = 0

    while True:
        # Send patient on the way
        action = generate_action(env, meanCT2register, meanCT2consult, receptionist, dietician, patient_ID)
        env.process(action)

        # Calculate the inter-arrival time before next patient arrives
        delta = random.expovariate(1.0 / mean_IAT)
        yield env.timeout(delta)

        # Next patient
        patient_ID += 1

### 3. Generate Action

Define the generator for registration and consulation.

**Args**:
- env (Simpy environment) required
- mean_CT2register (int) mean registration time
- mean_CT2consult (int) mean consultation time
- receptionist (Simpy resource) resource
- dietician (Simpy resource) resource
- patient_ID (int) patient ID from the function that generates arrivals

**Notes**:
- Has a with statement for registration to request a receptionist with which to timeout for sampled registration time. Will timeout until resource is available.
- Has a with statement for consultation to request a dietician with which to timeout for sampled consultation time. Will timeout until resource is available.
- Notes timestamps at: entered queue for registration, started registration, entered queue for consultation, started consultation, exited process.  

In [46]:
def generate_action(env, mean_CT2register, mean_CT2consult, receptionist, dietician, patient_ID):
    # timestamp queued for registration
    queued4registration = env.now

    # request receptionist
    with receptionist.request() as req:
        yield req

        # timestamp started registration
        startedRegistration = env.now
        print("{} started registration at {:.2f} after waiting {:.2f}".format(patient_ID, startedRegistration, 
                startedRegistration-queued4registration))

        # do! registration        
        delta_registration = random.expovariate(1.0 / mean_CT2register) 
        yield env.timeout(delta_registration)

    # timestamp queued for consultation
    queued4consultation = env.now

    # request dietician
    with dietician.request() as req:
        yield req
        
        # timestamp started consultation
        startedConsultation = env.now
        print("{} started consultation at {:.2f} after waiting {:.2f}".format(patient_ID, startedConsultation, 
                startedConsultation-queued4consultation))

        # do consultation
        delta_consultation = random.expovariate(1.0 / mean_CT2consult)
        yield env.timeout(delta_consultation)

    # timestamp exited
    exited = env.now
    print("{} left at {:.2f}".format(patient_ID, exited))

*Question*: The two ```with``` statements in ```generate_action()``` are at the same level. If one were nested within the other, how does the meaning change?

### 4. Set Up & Run Simulation

Define simulation variables as follows:
- Create an instance of env
- Initialize resources
- Define key parameters (for distributions)

In [47]:
# Set up the simulation environment
env = simpy.Environment()

# Set up the resources
receptionist = simpy.Resource(env, 1)
dietician = simpy.Resource(env, 1)

# Configure simulation parameters
mean_IAT = 5 
mean_CT2register = 6
mean_CT2consult = 8

In [48]:
env.process(generate_arrivals(env, mean_IAT, mean_CT2register, mean_CT2consult, receptionist, dietician))
env.run(until=120)

0 started registration at 0.00 after waiting 0.00
0 started consultation at 3.10 after waiting 0.00
1 started registration at 3.10 after waiting 1.88
0 left at 5.34
1 started consultation at 9.35 after waiting 0.00
2 started registration at 9.35 after waiting 4.01
1 left at 16.55
2 started consultation at 28.53 after waiting 0.00
3 started registration at 28.53 after waiting 19.62
2 left at 30.19
3 started consultation at 33.01 after waiting 0.00
4 started registration at 33.01 after waiting 19.71
3 left at 34.08
4 started consultation at 39.24 after waiting 0.00
5 started registration at 39.24 after waiting 23.48
6 started registration at 39.98 after waiting 19.98
7 started registration at 43.33 after waiting 22.77
8 started registration at 43.68 after waiting 22.86
4 left at 45.41
5 started consultation at 45.41 after waiting 5.43
5 left at 48.42
6 started consultation at 48.42 after waiting 5.09
6 left at 51.01
7 started consultation at 51.01 after waiting 7.33
9 started registratio

## Part B: Archive Data

So far we have printed out results. Now, let us save them using arrays. Define global arrays for saving timestamps. Then proceed to persist information to file.

We want to know how efficient is our process. Efficiency means minimizing wster. From a patient's vantage point, waste is time spent in queuing. We shall persist information about time spent in queuing and the lead time from arrival to exit.


### 1. Generate Arrivals

Modify the generator for arrivals to use a new generator function to process arrivals.

In [49]:
def generate_arrivals2(env, mean_IAT, meanCT2register, meanCT2consult, receptionist, dietician):
    # Create patient zero
    patient_ID = 0

    while True:
        # Send patient on the way
        action = generate_action2(env, meanCT2register, meanCT2consult, receptionist, dietician, patient_ID)
        env.process(action)

        # Calculate the inter-arrival time before next patient arrives
        delta = random.expovariate(1.0 / mean_IAT)
        yield env.timeout(delta)

        # Next patient
        patient_ID += 1

### 2. Generate Action

Modify the function to use globals to persist information.

In [50]:
def generate_action2(env, mean_CT2register, mean_CT2consult, receptionist, dietician, patient_ID):
    # globals
    global queued4registration
    global queued4consultation
    global leadTime
    
    # timestamp queued for registration
    arrived = env.now

    # request receptionist
    with receptionist.request() as req:
        yield req

        # timestamp started registration
        startedRegistration = env.now
        print("{} started registration at {:.2f} after waiting {:.2f}".format(patient_ID, startedRegistration, 
                startedRegistration-arrived))
        queued4registration.append(startedRegistration - arrived)

        # do! registration        
        delta_registration = random.expovariate(1.0 / mean_CT2register) 
        yield env.timeout(delta_registration)

    # timestamp queued for consultation
    enteredQueue4consultation = env.now

    # request dietician
    with dietician.request() as req:
        yield req
        
        # timestamp started consultation
        startedConsultation = env.now
        print("{} started consultation at {:.2f} after waiting {:.2f}".format(patient_ID, startedConsultation, 
                startedConsultation-enteredQueue4consultation))
        queued4consultation.append(startedConsultation-enteredQueue4consultation)

        # do consultation
        delta_consultation = random.expovariate(1.0 / mean_CT2consult)
        yield env.timeout(delta_consultation)

    # timestamp exited
    exited = env.now
    print("{} left at {:.2f}".format(patient_ID, exited))
    leadTime.append(exited-arrived)

### 3. Set Up & Run Simulation

Initialize the globals.

In [51]:
# Set up the simulation environment
env = simpy.Environment()

# Set up the resources
receptionist = simpy.Resource(env, 1)
dietician = simpy.Resource(env, 2)

# Configure simulation parameters
mean_IAT = 5 
mean_CT2register = 6
mean_CT2consult = 8

# Globals
queued4registration = []
queued4consultation = []
leadTime = []

In [52]:
# Make it so!
env.process(generate_arrivals2(env, mean_IAT, mean_CT2register, mean_CT2consult, receptionist, dietician))
env.run(until=240)

0 started registration at 0.00 after waiting 0.00
0 started consultation at 19.67 after waiting 0.00
1 started registration at 19.67 after waiting 19.54
0 left at 21.11
1 started consultation at 21.73 after waiting 0.00
2 started registration at 21.73 after waiting 11.32
1 left at 25.58
2 started consultation at 37.26 after waiting 0.00
3 started registration at 37.26 after waiting 19.95
3 started consultation at 40.62 after waiting 0.00
4 started registration at 40.62 after waiting 20.66
2 left at 43.47
4 started consultation at 44.62 after waiting 0.00
5 started registration at 44.62 after waiting 23.63
3 left at 45.88
5 started consultation at 47.16 after waiting 0.00
6 started registration at 47.16 after waiting 20.80
4 left at 47.53
6 started consultation at 47.77 after waiting 0.00
7 started registration at 47.77 after waiting 21.12
8 started registration at 47.96 after waiting 21.00
9 started registration at 48.79 after waiting 11.48
5 left at 52.64
7 started consultation at 52.

### 4. Review Results

In [53]:
# Persisted info.

leadTime

[21.10988473215878,
 25.45330925631963,
 33.0693918747596,
 28.575569081573306,
 27.570786503942852,
 31.642362275015834,
 29.288387954052723,
 30.283119068862096,
 19.727064070929266,
 31.42754915464842,
 22.0579431408594,
 36.52943221157606,
 41.32461265987846,
 43.60623906582316,
 47.69894770832763,
 45.446813461479735,
 43.44975575141555,
 55.190676141345115,
 47.72509920464553,
 52.28691318066912,
 50.67627968311875,
 49.08605042624876,
 45.24084094761261,
 44.61089747017755,
 49.172949247475984,
 52.55831785984651,
 50.61622014412791,
 51.34992932869615,
 48.98595470348033,
 58.62849139441978,
 60.14023952964378,
 68.21864539332918,
 70.69440284058626,
 71.56087524660381,
 88.22111848051085,
 85.0754116809454,
 87.37423662257726,
 90.50939730075478]